In [23]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os
import statistics
import seaborn as sns
from scipy.stats import wilcoxon
from scipy.stats import ranksums
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
from scipy.stats import false_discovery_control
from itertools import combinations
import matplotlib.pyplot as plt
import pickle
from statsmodels.stats.multitest import multipletests
import re

In [2]:
def p_adjust_fdr(p):
    """
    Benjamini-Hochberg p-value correction for multiple hypothesis testing.

    Parameters
    ----------
    p : ndarray, list
        Array or list of p-values to correct.

    Returns
    -------
    corr_p : ndarray
        Array of corrected p-values.
    """

    # Code adapted from: https://stackoverflow.com/a/33532498/8395875
    p = np.asarray(p, dtype='float64')
    by_descend = p.argsort()[::-1]
    by_orig = by_descend.argsort()
    steps = float(len(p)) / np.arange(len(p), 0, -1)
    q = np.minimum(1, np.minimum.accumulate(steps * p[by_descend]))
    corr_p = q[by_orig]

    return corr_p

In [3]:
#%pip install --upgrade decoupler

In [4]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    if arguments_list["comparison_list"] is None:
        arguments_list["comparison_list"] = np.nan

    if arguments_list["logfc"] is None:
        arguments_list["logfc"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list ["num_cell_filter"] is None:
        arguments_list["num_cell_filter"] = 0

    if arguments_list["reg"] is None:
        arguments_list["reg"] = load_dorothea_regulon(arguments_list["organism"])

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    #is the naming of tf fine like this?
    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight" in arguments_list["reg"]:
        raise NameException("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns 'source', 'target' and 'weight'!")
    
    if arguments_list["plot"] is None:
        arguments_list["plot"] = True
    elif not isinstance(arguments_list["plot"], (bool)):
        raise ValueError("lot argument must be a boolean value.")
        
   
    return(arguments_list)



In [5]:
class TFObj:
    def __init__(self,
    tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        self.tf_activities_condition = tf_activities_condition
        self.tf_activities_cluster = tf_activities_cluster
        self.average_gene_expression = average_gene_expression
        self.regulon = regulon
        self.CTR_input_condition = CTR_input_condition
        self.CTR_input_cluster = CTR_input_cluster
        self.intracellular_network_condition = intracellular_network_condition
        self.intracellular_network_cluster = intracellular_network_cluster

def make_TFOBj(tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        tf = TFObj(tf_activities_condition,
            tf_activities_cluster,
            average_gene_expression,
            regulon,
            CTR_input_condition,
            CTR_input_cluster,
            intracellular_network_condition,
            intracellular_network_cluster)

        return tf

In [6]:
def AverageExpression(sub_object, celltype = None, name_iterable = None, outpath = None):
    gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
    obs = sub_object[:,gene_ids].X.toarray()
    obs = np.expm1(obs)
    avg_df = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs[celltype])
    avg_df = avg_df.groupby(level=0, observed=False).mean()
    #avg_df.T.to_csv(outpath + name_iterable + "_average_gene_expression_by_cluster_exp.csv")

    return avg_df.T


In [7]:
def eval_pval(p_val):
    p_val = float(p_val)
    if p_val < 0.001: 
      txt = "***"
    elif p_val < 0.01: 
      txt = "**"
    elif p_val < 0.05: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)


def eval_log_fc_tag(log_fc):
    if log_fc >= 1.0: 
      txt = "***"
    elif log_fc > 0.5: 
      txt = "**"
    elif log_fc > 0.0: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)

In [8]:
def create_unfiltered_tf_scores(tf_scores_df, condition, celltype, out_path):   
    summarized_tf_scores_df = tf_scores_df.groupby(celltype, observed = True).mean().T
    #tf_scores_df.groupby(celltype, observed = True).apply(display)
    #agg(["mean", "var"])
    summarized_tf_scores_df.to_csv(out_path + "/unfiltered_tf_scores_" + condition + ".csv")
    return summarized_tf_scores_df

#np.var() returns a value that is different from R's var(), whereas statistics.variance() is the same as R's var()
def save_variable_tf_score(filtered_summarized_tf_scores_df, condition, out_path, plot):
    filtered_summarized_tf_scores_df["var"] = filtered_summarized_tf_scores_df.apply(statistics.variance, axis=1).unique()
    filtered_summarized_tf_scores_df.to_csv(out_path + "/variable_tf_scores_" + condition + ".csv")

    if plot:
        top_variable_tfs = filtered_summarized_tf_scores_df.sort_values("var", ascending=False).head(n=20).drop(columns="var")
        cluster_map = sns.clustermap(top_variable_tfs, cmap="vlag", center=0, vmin=top_variable_tfs.min(axis=None), cbar_kws={"label": "z-score"},
                                    cbar_pos=(1, 0.5, 0.02, 0.1), dendrogram_ratio=(0.2, 0.05))
        cluster_map.ax_heatmap.set_xlabel("Cell Type")
        cluster_map.ax_heatmap.set_ylabel("Transcription Factor")
        plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=90) 
        plt.savefig(out_path + "/tf_activity_top20_variable_" + condition + ".pdf", bbox_inches='tight')
        plt.close()

    filtered_summarized_tf_scores_df_var = filtered_summarized_tf_scores_df
    return filtered_summarized_tf_scores_df_var
    

In [9]:
def plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping, condition, out_path):
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.drop(columns="var")
    calc_size = ((len(filtered_summarized_tf_scores_df.columns.unique()) * 1.5), (len(filtered_summarized_tf_scores_df) * 0.035))
    cluster_map = sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "z-score"}, figsize=calc_size, cmap="vlag", center=0, 
                                yticklabels=False, cbar_pos=(1, 0.5, 0.02, 0.1), dendrogram_ratio=(0.2, 0.05))
    cluster_map.ax_heatmap.set_xlabel("Cell Type")
    cluster_map.ax_heatmap.set_ylabel("Transcription Factor")
    plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=90) 
    
    plt.savefig(out_path + "/tf_activity_compressed_" + condition + ".pdf", bbox_inches='tight')
    plt.close()

    calc_size = ((len(filtered_summarized_tf_scores_df.columns.unique()) * 1.5), (len(filtered_summarized_tf_scores_df) * 0.2))
    cluster_map = sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "z-score"}, figsize=calc_size, cmap="vlag", center=0, annot= tag_mapping, fmt="", 
                                yticklabels=True, cbar_pos=(1, 0.5, 0.03, 0.05), dendrogram_ratio=(0.2, 0.05))
    cluster_map.ax_heatmap.set_xlabel("Cell Type")
    cluster_map.ax_heatmap.set_ylabel("Transcription Factor")
    plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=90) 

    plt.savefig(out_path + "/tf_activity_" + condition + ".pdf", bbox_inches='tight')
    plt.close()


    

In [10]:
def h_clust(data):
            dist_matrix = pdist(data.T)
            linkage_matrix = linkage(dist_matrix, method = "average")   
            return linkage_matrix

In [11]:
def plot_condition_tf_activities(tf_activity_tables, out_path):
       
   for result_name, df in tf_activity_tables.items():
        name_df = pd.DataFrame(df)
        significant_res = name_df[name_df["tag"] == "***"]
        significant_genes = np.unique(significant_res["tf"])
        name_df = name_df[name_df['tf'].isin(significant_genes)]
        

        tag_mapping = name_df[["tf", "tag", "CellType"]]
        #print(tag_mapping)
        tag_mapping = tag_mapping.pivot(index="tf", columns="CellType", values="tag")
        tag_mapping.fillna("ns", inplace=True)
    
        name_df_r = name_df[["r", "tf", "CellType"]]
        name_df_cluster = name_df_r.pivot_table(index = "tf", columns = "CellType", values = "r", aggfunc = "mean")
        name_df_cluster.fillna(0, inplace=True) 
        
        h_clust_matrix = h_clust(name_df_cluster)
      
        calc_size = ((len(name_df["CellType"].unique()) * 1.5), (len(name_df_cluster) * 0.2))
      
        cluster_map = sns.clustermap(name_df_cluster, cbar_kws={"label": "r"}, figsize=calc_size, cmap="vlag", center=0, annot= tag_mapping,
                      col_linkage= h_clust_matrix, fmt="", yticklabels=True, cbar_pos=(1, 0.5, 0.03, 0.05), dendrogram_ratio=(0.2, 0.05))
        plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=90)  

        plt.savefig(out_path + "/" + result_name + "_cluster_condition_activity_difference.pdf", bbox_inches='tight')
        plt.close()

In [12]:
def plot_condition_tf_activities_compressed(tf_activity_tables, out_path):
 
  
    for result_name, df in tf_activity_tables.items():
        name_df = pd.DataFrame(df)
        
        significant_res = name_df[name_df["tag"] == "***"]
        significant_genes = np.unique(significant_res["tf"])
        name_df = name_df[name_df['tf'].isin(significant_genes)]
        
        name_df_r = name_df[["r", "tf", "CellType"]]
        name_df_cluster = name_df_r.pivot_table(index = "tf", columns = "CellType", values = "r", aggfunc = "mean")
        name_df_cluster.fillna(0, inplace=True) 

        h_clust_matrix = h_clust(name_df_cluster)
        name_df_cluster.reset_index()

        calc_size = ((len(name_df["CellType"].unique()) * 1.5), (len(name_df_cluster) * 0.05))
        cluster_map = sns.clustermap(name_df_cluster, cbar_kws={"label": "r"}, figsize=calc_size, cmap="vlag", center=0, annot= None,
                       yticklabels=False, col_linkage= h_clust_matrix, fmt="", cbar_pos=(1, 0.5, 0.02, 0.1), dendrogram_ratio=(0.2, 0.05))
        plt.setp(cluster_map.ax_heatmap.get_xticklabels(), rotation=90) 
        cluster_map.figure.suptitle(result_name)
        
        
        plt.savefig(out_path + "/" + result_name +  "_cluster_condition_activity_difference_compressed.pdf", bbox_inches='tight')
        plt.close()

In [13]:
def map_z_value(tf_scores_df, anndataobject_markers):
    genes = []
    clusters = []
    z_scores = []  

    anndataobject_markers = anndataobject_markers.set_index("gene")
    
    for i in range(len(anndataobject_markers.index)):
        a = anndataobject_markers.index[i]
        for j in range(len(tf_scores_df.index)):
            b = tf_scores_df.index[j]
            if a == b:
                c = anndataobject_markers.columns.get_loc("cluster")
                cluster = anndataobject_markers.iloc[i, c]
                gene_rows = tf_scores_df.iloc[j]
                if isinstance(gene_rows, pd.Series):
                    gene_rows = gene_rows.to_frame().T

                for _, gene_row in gene_rows.iterrows():
                     score = gene_row[cluster]
                     genes.append(a)
                     clusters.append(cluster)
                     z_scores.append(score)

    z_score_df = pd.DataFrame({
        'gene': genes,
        'cluster': clusters,
        'z_score': z_scores
    })

    return z_score_df

In [14]:
def add_entry(source, target, gene_A, gene_B, type_gene_A, type_gene_B, MeanLR):
  df = {"source" : source,
      "target" : target,
      "gene_A" : gene_A,
      "gene_B" : gene_B,
      "type_gene_A" : type_gene_A,
      "type_gene_B" : type_gene_B,
      "MeanLR" : MeanLR}

  return df

In [15]:
def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None, organism = "human"):

  if organism == "human":
    ligands = pd.read_csv("ligands_human.csv")
    R2TF = pd.read_csv("rtf_db_human.csv")
  elif organism == "mouse": 
    ligands = pd.read_csv("ligands_mouse.csv")
    R2TF = pd.read_csv("rtf_db_mouse.csv")
  else:
    NameError("Invalid organism to generate CrossTalkeR input!")

  R2TF = R2TF.set_index("tf")

  sorted_regulon = regulon[["tf", "target"]]
  sorted_regulon = sorted_regulon.set_index("tf")


  output_list = []
  #print("gene epr index", gene_expression.index)

  for row in range(len((tf_activities))):
    
    tf_l = []
    r_tf = []

    #if (tf_activities["z_score"].iloc[row] > 0):
    tf = str(tf_activities["gene"].iloc[row])
    #print("tf", tf)
    if tf in sorted_regulon.index:
      targets = sorted_regulon.loc[tf, "target"]
      #print("targets", targets)
   
    if tf in R2TF.index:
        receptors = R2TF.loc[tf, "receptor"]
        if isinstance(receptors, str):
          receptors = [receptors]
       
    else:
        receptors = []

    

    tf_ligands = np.intersect1d(targets, ligands)
    
  #print("tf ligands", tf_ligands)

    if organism == "human":
      if len(tf_ligands) > 0:
        for ligand in tf_ligands:
            #print("ligand", ligand)
            expressed = False
            if ligand in gene_expression.index:
              ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, 2]]
              #print("ex value", ex_value)
              if (ex_value != 0):
                expressed = True

            if (expressed == True):
              df_list_l = add_entry(source = tf_activities.iloc[row, 2],
                                                      target = tf_activities.iloc[row, 2],
                                                      gene_A =tf_activities.iloc[row, 0],
                                                      gene_B = ligand,
                                                      type_gene_A = "Transcription_Factor",
                                                      type_gene_B= "Ligand",
                                                      MeanLR= tf_activities.iloc[row, 3]
                                                      )
              
            #print("df list l", df_list_l)  
              output_list.append(df_list_l)
              

      if (len(receptors) > 0):
        for receptor in receptors:
          #print("receptor", receptor)
          df_list_r = add_entry(source = tf_activities.iloc[row, 2],
                                                target = tf_activities.iloc[row, 2],
                                                gene_A= receptor,
                                                gene_B= tf_activities.iloc[row, 0],
                                                type_gene_A= "Receptor",
                                                type_gene_B= "Transcription Factor",
                                                MeanLR= tf_activities.iloc[row, 3]
                                                )
          #print("df list r", df_list_r)
          output_list.append(df_list_r)

      
          
    else: 
      if len(tf_ligands) > 0:
        for ligand in tf_ligands:
            expressed = False
            translations = ligand
            if len(translations) > 0:
              for l in translations:
                #print(gene_expression.index)
                if l in gene_expression.index:
                  ex_value = gene_expression.loc[l, tf_activities.iloc[row, 2]]
                  if (ex_value != 0):
                    expressed = True
          
                df_list_l = []
            
                if (expressed == True):
                  df_list_l = add_entry(source = tf_activities.iloc[row, 2],
                                                      target = tf_activities.iloc[row, 2],
                                                      gene_A =tf_activities.iloc[row, 0],
                                                      gene_B = ligand,
                                                      type_gene_A = "Transcription_Factor",
                                                      type_gene_B= "Ligand",
                                                      MeanLR= tf_activities.iloc[row, 3]
                                                      )
                
                  output_list.append(df_list_l)
      
      if (len(receptors) > 0):
        for receptor in receptors:
          df_list_r = [] 
          df_list_r = add_entry(tf_activities.iloc[row, 2],
                                                tf_activities.iloc[row, 2],
                                                receptor,
                                                tf_activities.iloc[row, 0],
                                                "Receptor",
                                                "Transcription Factor",
                                                tf_activities.iloc[row, 3]
                                                )
                                                
          output_list.append(df_list_r)
        #tf_l.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_l.csv", index=0)
        #r_tf.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_r.csv", index=0)
   

  output_df = pd.DataFrame(output_list)
  output_df["gene_A"] = output_df["gene_A"].apply(lambda x: re.sub("_", "+", x))
  output_df["gene_B"] = output_df["gene_B"].apply(lambda x: re.sub("_", "+", x))
  output_df.to_csv("tf_l_r.csv", index=0)


  return output_df

In [16]:
def generate_intracellular_network(tf_activities, gene_expression, outpath, regulon, organism="human"):

    if len(tf_activities.shape) > 0:
        if organism == "human":
            R2TF = pd.read_csv("rtf_db_human.csv").set_index("tf")
        else:
            R2TF = pd.read_csv("rtf_db_mouse.csv").set_index("tf")

    sorted_regulon = regulon[["tf", "target"]].set_index("tf")

    #preextract values
    tf_genes = tf_activities["gene"].values
    tf_celltypes = tf_activities.iloc[:, 2].values
    tf_scores = tf_activities.iloc[:, 3].values

    TFTG_list = []
    RTF_list = []

    for row in range(len(tf_activities)):
        tf = str(tf_genes[row])
        celltype = tf_celltypes[row]
        tf_score = tf_scores[row]

        targets = sorted_regulon.loc[tf, "target"] if tf in sorted_regulon.index else []
        receptors = R2TF.loc[tf, "receptor"] if tf in R2TF.index else []

        if len(targets) > 0 and len(receptors) > 0:
            for target in targets:
                if target in gene_expression.index:
                    ex_value = gene_expression.at[target, celltype]
                    if ex_value != 0:
                        TFTG_list.append({
                            "celltype": celltype,
                            "TF": tf,
                            "Target_Gene": target,
                            "TF_Score": tf_score
                        })

            for receptor in receptors:
                RTF_list.append({
                    "TF": tf,
                    "Receptor": receptor
                })

    TFTG_df = pd.DataFrame(TFTG_list)
    RTF_df = pd.DataFrame(RTF_list)

    recept_regulon = pd.merge(RTF_df, TFTG_df, on="TF")

    return recept_regulon

In [17]:
#decoupler condition comparison
def condition_comparison_significant(tf_activities, out_path, celltype, condition, comparison_list):

    tf_activities = sc.pp.scale(tf_activities, copy= True)
    
    vs_df_dic = {}

    if isinstance(comparison_list[0], str):  
        comparison_list = [comparison_list]
    
    for vs1, vs2 in comparison_list:

        print(f"vs1: {vs1}, vs2: {vs2}") 

        all_tf_list = tf_activities.var_names

        res = pd.DataFrame()
        for i in tf_activities.obs[celltype].unique(): 
            comparison_sub = tf_activities[(tf_activities.obs[celltype] == i) & (tf_activities.obs[condition].isin([vs1, vs2]))]
            if len(pd.unique(comparison_sub.obs[condition])) == 2:
                condition_table = comparison_sub.obs[[condition]].copy()
                condition_table.columns = ["condition"]
                metadata_counts = condition_table.groupby("condition", observed = False).size()
                
                if (metadata_counts.iloc[0] + metadata_counts.iloc[1]) > 10:
                    g = comparison_sub.obs[condition].astype("category")
                    g = g.cat.set_categories([vs1, vs2])
                    
                    #scanpy
                    #sc.tl.rank_genes_groups(comparison_sub, groupby= condition,
                     #                    reference="rest", method="wilcoxon", key_added="condition_comp_markers", corr_method= "bonferroni")
                    
                    #sc.pl.rank_genes_groups_heatmap(comparison_sub, show_gene_labels=True, key="condition_comp_markers")

                    #res_tmp = sc.get.rank_genes_groups_df(comparison_sub, group = None, log2fc_min=0, key="condition_comp_markers")
                    #######

                    #decoupler
                    comparison_sub = sc.pp.scale(comparison_sub, copy= True)
                    res_tmp = dc.rank_sources_groups(comparison_sub, groupby= condition, reference="rest", method="wilcoxon")
                    res_tmp.rename(columns={"group": "condition", "statistic" : "scores", "meanchange" : "logfoldchanges"}, inplace=True)
            
               

                    group1 = comparison_sub.X[g == vs1]
                    group2 = comparison_sub.X[g == vs2]
                        
                    
                    res_tmp["r"] = res_tmp["scores"] / np.sqrt(len(group1) + len(group2))
                    res_tmp["CellType"] = i
                    res_tmp["FDR"] = multipletests(res_tmp["pvals"], alpha=0.05, method='fdr_bh')
                    
                    
                    res_tmp["logfoldchanges"] = res_tmp["logfoldchanges"].where(res_tmp["logfoldchanges"] > 0, np.nan) 

                    res = pd.concat([res, res_tmp], ignore_index=True)

        res_df = res.dropna()

        def assign_significance_tag(fdr):
            if fdr < 0.001:
                return "***"
            elif fdr < 0.01:
                return "**"
            elif fdr < 0.05:
                return "*"
            else:
                return "ns"

        res_df["tag"] = res_df["FDR"].apply(assign_significance_tag)
        
        res_df.rename(columns={"names":"tf", "group": "condition"}, inplace=True)
        res_df.to_csv(f"{out_path}/all_tfs_{vs1}_vs_{vs2}.csv", index=False)

        result_name = f"{vs1}_{vs2}"
        vs_df_dic[result_name] = res_df
        #print(vs_df_dic[result_name])
    return vs_df_dic

In [18]:
#decoupler significant tfs

def get_significant_tfs(tf_activities_sub, condition, out_path, tf_condition_significant, celltype, pval, logfc, plot, condition_comparison = False):
    
    renamed_condition = condition.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)
        
    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_scaled, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 
    anndataobject_markers_wilcoxon = dc.rank_sources_groups(tf_activities_scaled, groupby= "new_annotation", reference="rest", method="wilcoxon")

    anndataobject_markers_wilcoxon.rename(columns={"names" : "gene", "group": "cluster", "statistic" : "scores", "meanchange" : "logfoldchanges"}, inplace=True)
    
    #
    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    
    print("after concat", anndataobject_markers_wilcoxon)
    
    anndataobject_markers_wilcoxon["tag"] = anndataobject_markers_wilcoxon["pvals_adj"].apply(eval_pval)
    anndataobject_markers_wilcoxon["log_fc_tag"] = anndataobject_markers_wilcoxon["logfoldchanges"].apply(eval_log_fc_tag)
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_wilcoxon_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval))] 
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox = tag_mapping_wilcox.astype("object")
    tag_mapping_wilcox.fillna("ns", inplace=True)
    
    #makes the index of the subobject not unique --- not possible to filter by index after this
    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    #print(tag_mapping_wilcox)
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    #print(col_num)
    filtered_tf_scores_df = tf_scores_df.loc[:, sorted(col_num)]


    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f"{single_result_path}/tf_scores_{condition}.csv")
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    filtered_summarized_tf_scores_df.index = filtered_summarized_tf_scores_df.index.map(lambda x: re.sub(".,", "_", x))
   
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.where(filtered_summarized_tf_scores_df > 0, np.nan) 
    
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/significant_cluster_tf_results_wilcoxon_" + renamed_condition + ".csv", index=0)
    #res_t_test.to_csv(single_result_path  + "/_significant_cluster_tf_results_t_test_" + renamed_condition + ".csv", index=0)

    res = {}
    res["cluster"] = res_wilcoxon
    
    tf_condition_significant["gene"] = tf_condition_significant["gene"].apply(lambda x: re.sub(".,", "_", x))

    if condition_comparison:
        unfiltered_tf_scores = unfiltered_tf_scores.where(unfiltered_tf_scores > 0, np.nan) 
        tf_condition_significant = tf_condition_significant.merge(map_z_value(unfiltered_tf_scores, tf_condition_significant), left_on=None, right_on=None, left_index=False, right_index=False)
        tf_condition_significant.dropna(inplace=True)
    
        res["condition"] = tf_condition_significant
        res["condition"].to_csv(f"{single_result_path}/significant_condition_tf_results_{condition}.csv", index=0)

    return res

In [19]:
def condition_comparison_significant(tf_activities, out_path, celltype, condition, comparison_list):
    
    tf_activities = sc.pp.scale(tf_activities, copy= True)

    vs_df_dic = {}
          
    if isinstance(comparison_list[0], str):  
        comparison_list = [comparison_list]
    
    for vs1, vs2 in comparison_list:

        print(f"vs1: {vs1}, vs2: {vs2}") 

        all_tf_list = tf_activities.var_names

        res = {}
        for i in tf_activities.obs[celltype].unique(): 
            comparison_sub = tf_activities[(tf_activities.obs[celltype] == i) & (tf_activities.obs[condition].isin([vs1, vs2]))]
            if len(pd.unique(comparison_sub.obs[condition])) == 2:
                condition_table = comparison_sub.obs[[condition]].copy()
                condition_table.columns = ["condition"]
                metadata_counts = condition_table.groupby("condition", observed = False).size()
                
                if (metadata_counts.iloc[0] + metadata_counts.iloc[1]) > 10:
                    g = comparison_sub.obs[condition].astype("category")
                    g = g.cat.set_categories([vs1, vs2])
                    
                    #scanpy
                    #sc.tl.rank_genes_groups(comparison_sub, groupby= condition,
                     #                    reference="rest", method="wilcoxon", key_added="condition_comp_markers", corr_method= "bonferroni")
                    
                    #sc.pl.rank_genes_groups_heatmap(comparison_sub, show_gene_labels=True, key="condition_comp_markers")

                    #res_tmp = sc.get.rank_genes_groups_df(comparison_sub, group = None, log2fc_min=0, key="condition_comp_markers")
                    #######

                    #decoupler
                    #comparison_sub = sc.pp.scale(comparison_sub, copy= True)
                    #res_tmp = dc.rank_sources_groups(comparison_sub, groupby= condition, reference="rest", method="wilcoxon")
                    #res_tmp.rename(columns={"group": "condition", "statistic" : "scores", "meanchange" : "logfoldchanges"}, inplace=True)
                    #####

                    res[i] = {}
                    for tf in all_tf_list:
                        group1 = comparison_sub[:, tf].X[g == vs1]
                        group2 = comparison_sub[:, tf].X[g == vs2]

                        if len(group1) > 0 and len(group2) > 0:
                            # Wilcoxon test
                            stat, pval = ranksums(group1, group2)
                            
                            r = abs(stat) / np.sqrt(len(group1) + len(group2))  
                            l2fc = np.log2((group1.mean()+ 1e-6)/(group2.mean() + 1e-6))
                            res[i][tf] = {"tf": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : float(l2fc), "CellType" : i}

                    res[i] = pd.DataFrame.from_dict(res[i], orient="index")
                    
                    
        res_df = pd.concat(res.values(), axis=0, ignore_index=True)

        #default is Benjamini Hochberg correction
        _, res_df["FDR"], _, _ = multipletests(res_df["pval"], alpha=0.05, method='fdr_bh')
        res_df = res_df.dropna()

        def assign_significance_tag(fdr):
            if fdr < 0.001:
                return "***"
            elif fdr < 0.01:
                return "**"
            elif fdr < 0.05:
                return "*"
            else:
                return "ns"

        res_df["tag"] = res_df["FDR"].apply(assign_significance_tag)
        
        res_df.rename(columns={"names":"tf", "group": "condition"}, inplace=True)
        res_df.to_csv(f"{out_path}/all_tfs_{vs1}_vs_{vs2}.csv", index=False)

        result_name = f"{vs1}_{vs2}"
        vs_df_dic[result_name] = res_df
        #print(vs_df_dic[result_name])
    return vs_df_dic

In [20]:
def get_significant_tfs(tf_activities_sub, condition, out_path, tf_condition_significant, celltype, pval, logfc, plot, condition_comparison = False):
    
    renamed_condition = condition.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)
    #zero center = False is possible and delivers results for rank genes groups, but then the tf scores wont be the same as in R

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_sub, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 

    #decoupler
    #anndataobject_markers_wilcoxon = dc.rank_sources_groups(tf_activities_scaled, groupby= "new_annotation", reference="rest", method="wilcoxon")
    
    #scanpy
    #sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")

    #sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers", corr_method= "bonferroni" )
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")

    #anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="wilcoxon_markers")
    #print(anndataobject_markers_wilcoxon)

    #scipy
    res = {}
    all_tf_list = tf_activities_sub.var_names
    for i in tf_activities_scaled.obs[celltype].unique(): 
        res[i] = {}

        group1_mask = tf_activities_sub.obs[celltype] == i
        group2_mask = tf_activities_sub.obs[celltype] != i
        
        for tf in all_tf_list:
            group1 = tf_activities_sub[group1_mask, tf].X
            group2 = tf_activities_sub[group2_mask, tf].X

            #print("g1", group1)
            #print("g2", group2)

            mean1 = np.mean(group1) if len(group1) > 0 else 0
            mean2 = np.mean(group2) if len(group2) > 0 else 0
            
            if len(group1) > 0 and len(group2) > 0:
                # Wilcoxon test
                stat, pval = ranksums(group1, group2)
                            
                r = abs(stat) / np.sqrt(len(group1) + len(group2))  
                l2fc = np.log2((group1.mean()+ 1e-6)/(group2.mean() + 1e-6))
                    
                res[i][tf] = {"gene": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : l2fc, "cluster" : i}

        res[i] = pd.DataFrame.from_dict(res[i], orient="index")
        #print("res",res[i])
                    
                    
    anndataobject_markers_wilcoxon = pd.concat(res.values(), axis=0, ignore_index=True)
    

    #default scipy false discovery control is Benjamini Hochberg correction
    _, anndataobject_markers_wilcoxon["pvals_adj"], _, _ = multipletests(anndataobject_markers_wilcoxon["pval"], alpha=0.05, method='fdr_bh')
    anndataobject_markers_wilcoxon = anndataobject_markers_wilcoxon.dropna()


    #####################scipy end

    #anndataobject_markers_wilcoxon.rename(columns={"names": "gene", "group": "cluster", "meanchange": "logfoldchanges"}, inplace=True)

    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    
    print("after concat", anndataobject_markers_wilcoxon)
    
    anndataobject_markers_wilcoxon["tag"] = anndataobject_markers_wilcoxon["pvals_adj"].apply(eval_pval)
    anndataobject_markers_wilcoxon["log_fc_tag"] = anndataobject_markers_wilcoxon["logfoldchanges"].apply(eval_log_fc_tag)
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_wilcoxon_test.csv",index=0)

    #anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
 
    #anndataobject_markers_t_over.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)

    #anndataobject_markers_t_over.loc[:, "tag"] = None
    #anndataobject_markers_t_over.loc[:, "log_fc_tag"] = None

    #for i in range(len(anndataobject_markers_t_over["pvals_adj"])):
    #     anndataobject_markers_t_over.loc[i, "tag"] = eval_pval(anndataobject_markers_t_over.loc[i, "pvals_adj"])
    #     anndataobject_markers_t_over.loc[i, "log_fc_tag"] = eval_pval(anndataobject_markers_t_over.loc[i, "logfoldchanges"])

    #anndataobject_markers_t_over.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_t_test_overestim_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval))] 
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox = tag_mapping_wilcox.astype("object")
    tag_mapping_wilcox.fillna("ns", inplace=True)
    
    #makes the index of the subobject not unique --- not possible to filter by index after this
    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    #print(tag_mapping_wilcox)
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    #print(col_num)
    filtered_tf_scores_df = tf_scores_df.loc[:, sorted(col_num)]


    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f"{single_result_path}/tf_scores_{condition}.csv")
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    filtered_summarized_tf_scores_df.index = filtered_summarized_tf_scores_df.index.map(lambda x: re.sub(".,", "_", x))
   
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.where(filtered_summarized_tf_scores_df > 0, np.nan) 
    
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/significant_cluster_tf_results_wilcoxon_" + renamed_condition + ".csv", index=0)
    #res_t_test.to_csv(single_result_path  + "/_significant_cluster_tf_results_t_test_" + renamed_condition + ".csv", index=0)

    res = {}
    res["cluster"] = res_wilcoxon
    
    tf_condition_significant["gene"] = tf_condition_significant["gene"].apply(lambda x: re.sub(".,", "_", x))

    if condition_comparison:
        unfiltered_tf_scores = unfiltered_tf_scores.where(unfiltered_tf_scores > 0, np.nan) 
        tf_condition_significant = tf_condition_significant.merge(map_z_value(unfiltered_tf_scores, tf_condition_significant), left_on=None, right_on=None, left_index=False, right_index=False)
        tf_condition_significant.dropna(inplace=True)
    
        res["condition"] = tf_condition_significant
        res["condition"].to_csv(f"{single_result_path}/significant_condition_tf_results_{condition}.csv", index=0)

    return res

In [21]:
def IntraTalker_analysis(anndataobject, tf_activities = None, arguments_list = None):
    
    if (isinstance(anndataobject, str)):
        anndataobject = ad.read_h5ad(anndataobject)

    arguments_list = validate_input_arguments(arguments_list)

    if arguments_list["decoupler_matrix_format"] == "R":
        anndataobject = anndataobject.T

    if not os.path.isdir(arguments_list["out_path"]):
        os.mkdir(arguments_list["out_path"])
        tf_path = arguments_list["out_path"] + "TF_results/"
        os.mkdir(tf_path)
    else:
        tf_path = arguments_list["out_path"] + "TF_results/"


    condition = anndataobject.obs[arguments_list["condition"]]

    if isinstance(tf_activities, str):
         tf_activities = ad.read_csv(tf_activities)
         tf_activities.obs = anndataobject.obs.reindex(tf_activities.obs.index)
         tf_activities.obsm = anndataobject.obsm
         tf_activities.uns = anndataobject.uns

    elif tf_activities is None:
         raise NameError("Please attach a csv file with the tf activity values. (For further clarification view the 'Decoupler' section of the vignette.)")
        

    #sets the stage for decision if single condition or comparison analysis is done

    if (len(arguments_list["comparison_list"]) > 0) & (len(pd.unique(anndataobject.obs[arguments_list["condition"]])) < 2):
        arguments_list["comparison_list"] = np.nan
        print("Only one condition was found in the data, although a list of comparisons was provided. The analyses are performed only for the present condition!")

    #code for single condition  analysis

    if pd.isnull(arguments_list["comparison_list"][0]):

        result_list = {}
        gene_expression_list = {}
        CTR_cluster_list = {}
        intranet_cluster_list = {}

        #creates loop until after tf activity score
        
        for name_iterable in anndataobject.obs[arguments_list["condition"]].unique():
            sub_object = anndataobject[anndataobject.obs[arguments_list["condition"]] == name_iterable].copy()
            tf_activities_sub = tf_activities[tf_activities.obs[arguments_list["condition"]] == name_iterable].copy()

            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], outpath= arguments_list["out_path"])
        
            
            tf_activity_scores = get_significant_tfs(tf_activities_sub,
                                                        name_iterable,
                                                        tf_path,
                                                        None,
                                                        celltype = arguments_list["celltype"],
                                                        pval = arguments_list["pval"],
                                                        logfc = arguments_list["logfc"],
                                                        plot = arguments_list["plot"],
                                                        condition_comparison= False)
            

            result_list[name_iterable] = tf_activity_scores
            
            gene_expression_list[name_iterable] = sub_object_avg
            
        
    
            CTR_cluster_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["cluster"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"],
                                                                            arguments_list["organism"])
    
              
            intranet_cluster_list[name_iterable] = generate_intracellular_network(tf_activity_scores["cluster"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
            

        tf = make_TFOBj(
                tf_activities_condition = list(),
                tf_activities_cluster = result_list,
                average_gene_expression = gene_expression_list,
                regulon = arguments_list["reg"],
                CTR_input_condition = list(),
                CTR_input_cluster = CTR_cluster_list,
                intracellular_network_condition = list(),
                intracellular_network_cluster = intranet_cluster_list)

        with open((arguments_list["out_path"] + "tf.pickle"), "wb") as file:
            pickle.dump(tf, file)

        return tf

    else:
        out_path_compared = (tf_path + "compared")
        if not os.path.isdir(out_path_compared):
            os.mkdir(out_path_compared)

        compared_significant_tfs = condition_comparison_significant(tf_activities, out_path_compared, arguments_list["celltype"], 
                                                                    arguments_list["condition"], arguments_list["comparison_list"])
        
        print("compared tfs done")
        
        if arguments_list["plot"] == True:
            plot_condition_tf_activities(compared_significant_tfs, out_path_compared)
            plot_condition_tf_activities_compressed(compared_significant_tfs, out_path_compared)

    
        result_condition_list = {}
        result_cluster_list = {}
        gene_expression_list = {}
        CTR_condition_list = {}
        CTR_cluster_list = {}
        intranet_condition_list = {}
        intranet_cluster_list = {}

        for name_iterable in anndataobject.obs[arguments_list["condition"]].unique():
            sub_object = anndataobject[anndataobject.obs[arguments_list["condition"]] == name_iterable]
            tf_activities_sub = tf_activities[tf_activities.obs[arguments_list["condition"]] == name_iterable]
            

            compared_tfs = pd.DataFrame({"gene" : pd.Series(dtype="str"), "tag" : pd.Series(dtype="str"), "cluster" : pd.Series(dtype="str")})
            #print("before", compared_significant_tfs)
        
            for result_name, df in compared_significant_tfs.items(): 
                if name_iterable in result_name:
                    tf_condition_significant = compared_significant_tfs[result_name]
                    tf_condition_significant = tf_condition_significant[tf_condition_significant["FDR"] < arguments_list["pval"]]
                    tf_condition_significant = tf_condition_significant[(tf_condition_significant["logfoldchanges"] > float(arguments_list["logfc"])) | (tf_condition_significant["logfoldchanges"] < (0 - float(arguments_list["logfc"])))]
                    tf_condition_significant = tf_condition_significant[["tf", "tag", "CellType"]]
                    tf_condition_significant.rename(columns={"tf":"gene", "CellType": "cluster"}, inplace=True)
                    compared_tfs = pd.concat([compared_tfs, tf_condition_significant])

        
            
            re.sub("([,;.:-])", "_", name_iterable)

            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], 
                                               outpath= arguments_list["out_path"])
            
            tf_activity_scores = get_significant_tfs(tf_activities_sub,
                                               name_iterable,
                                               tf_path,
                                               compared_tfs,
                                               celltype = arguments_list["celltype"],
                                               pval = arguments_list["pval"],
                                               logfc = arguments_list["logfc"],
                                               plot = arguments_list["plot"],
                                               condition_comparison= True)
            
            print("tf_activities done")

            result_condition_list[name_iterable] = tf_activity_scores["condition"]
            result_cluster_list[name_iterable] = tf_activity_scores["cluster"]

            #print(result_condition_list[name_iterable])
            #print(result_cluster_list[name_iterable])

            gene_expression_list[name_iterable] = sub_object_avg
            
        
            CTR_condition_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["condition"],
                                                                            gene_expression_list[name_iterable],
                                                                           arguments_list["out_path"],                                             
                                                                           arguments_list["reg"],
                                                                           arguments_list["organism"])
            
            print("CTR input condition done")
    
            CTR_cluster_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["cluster"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"],
                                                                            arguments_list["organism"])
    
            print("CTR input cluster done")

            intranet_condition_list[name_iterable] = generate_intracellular_network(tf_activity_scores["condition"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
    
            print("intranet input condition done")

            intranet_cluster_list[name_iterable] = generate_intracellular_network(tf_activity_scores["cluster"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
            print("intranet input cluster done")
            #print(CTR_cluster_list[name_iterable])
            #print(CTR_condition_list[name_iterable])
        tf = make_TFOBj(
                tf_activities_condition = result_condition_list,
                tf_activities_cluster = result_cluster_list,
                average_gene_expression = gene_expression_list,
                regulon = arguments_list["reg"],
                CTR_input_condition = CTR_condition_list,
                CTR_input_cluster = CTR_cluster_list,
                intracellular_network_condition = intranet_condition_list,
                intracellular_network_cluster = intranet_cluster_list)

        with open((arguments_list["out_path"] + "tf.pickle"), "wb") as file:
            pickle.dump(tf, file)
        
        return tf
            

In [24]:
result = IntraTalker_analysis(anndataobject= "LR2TF_test_run/anndata_object.h5ad", 
                              tf_activities= "decoupler_results.csv",
                              arguments_list= {"out_path" : "script_test", 
                                                "celltype" : "new_annotation",
                                                "condition" : "protocol", 
                                                "organism" : "human", 
                                                "comparison_list" : ["PMF,MF2", "control"], #["control", "PMF,MF2"]], 
                                                "logfc" : "0.5",
                                                "pval" : None, 
                                                "num_cell_filter": None,
                                                "reg" : "filterd_regulon.csv",                                                                                              
                                                "plot" : True,
                                                "decoupler_matrix_format" : "Python"})


vs1: PMF,MF2, vs2: control


C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2875754451.py:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i][tf] = {"tf": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : float(l2fc), "CellType" : i}
C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2875754451.py:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i][tf] = {"tf": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : float(l2fc), "CellType" : i}
C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2875754451.py:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a

compared tfs done


C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2328833940.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i][tf] = {"gene": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : l2fc, "cluster" : i}
C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2328833940.py:55: RuntimeWarning: invalid value encountered in log2
  l2fc = np.log2((group1.mean()+ 1e-6)/(group2.mean() + 1e-6))
C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2328833940.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i][tf] = {"gene": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : l2fc, "cluster" : i}
C:\Users\Larissa\AppData\Loca

after concat         gene          pval         r  logfoldchanges        cluster  \
0       ADNP  2.431538e-05  0.264333        0.601955  Megakaryocyte   
5       ARNT  4.104313e-02  0.127944        0.821594  Megakaryocyte   
6      ASCL1  1.018965e-06  0.306095        0.902508  Megakaryocyte   
8       ATF2  3.362491e-01  0.060218       -0.265139  Megakaryocyte   
9       ATF3  8.163960e-06  0.279349        3.088806  Megakaryocyte   
...      ...           ...       ...             ...            ...   
1789  ZNF740  9.804630e-13  0.446700        3.936125         Neural   
1790  ZNF750  2.126506e-06  0.296894       -1.440501         Neural   
1791  ZNF766  5.343028e-07  0.313962       -1.355708         Neural   
1792   ZNF83  3.040855e-01  0.064358       -0.304973         Neural   
1793   ZNF92  3.405578e-01  0.059683       -0.091260         Neural   

         pvals_adj   tag log_fc_tag  
0     9.130987e-05  None       None  
5     7.229875e-02  None       None  
6     5.052605e-06  

...

tf_activities done
CTR input condition done
CTR input cluster done
intranet input condition done
intranet input cluster done


C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2328833940.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i][tf] = {"gene": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : l2fc, "cluster" : i}
C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2328833940.py:55: RuntimeWarning: invalid value encountered in log2
  l2fc = np.log2((group1.mean()+ 1e-6)/(group2.mean() + 1e-6))
C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2328833940.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i][tf] = {"gene": tf,"pval": float(pval), "r": float(r), "logfoldchanges" : l2fc, "cluster" : i}
C:\Users\Larissa\AppData\Loca

after concat         gene          pval         r  logfoldchanges  cluster  pvals_adj  \
0       ADNP  7.468469e-07  0.317464       -1.824760   Neural   0.000005   
1        AHR  5.364299e-02  0.123791        1.992706   Neural   0.089322   
2         AR  6.056465e-01  0.033121        0.254889   Neural   0.672732   
3      ARID2  2.063456e-04  0.238068        1.169711   Neural   0.000739   
5       ARNT  8.104540e-01  0.015386        0.113965   Neural   0.848259   
...      ...           ...       ...             ...      ...        ...   
1790  ZNF750  1.471466e-01  0.092997        0.384183  Myeloid   0.211134   
1791  ZNF766  4.465581e-03  0.182395        1.191781  Myeloid   0.010547   
1792   ZNF83  5.875000e-01  0.034799       -0.357529  Myeloid   0.655415   
1793   ZNF92  6.808322e-03  0.173595        0.584972  Myeloid   0.015185   
1794    ZZZ3  2.905725e-01  0.067798       -2.132655  Myeloid   0.370439   

       tag log_fc_tag  
0     None       None  
1     None       None  
2 

C:\Users\Larissa\AppData\Local\Temp\ipykernel_29460\2328833940.py:101: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval))]
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\anndata\_core\anndata.py:841: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    ['Neural', 'MSC', 'Megakaryocyte', 'Neural', 'Neural']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "obs")


tf_activities done
CTR input condition done
CTR input cluster done
intranet input condition done
intranet input cluster done


In [108]:
result.CTR_input_condition["control"].to_csv("py_ctr_input_wo_ctr_exp_tables.csv")
result.intracellular_network_condition["control"].to_csv("py_intra_network_ctrl.csv")
result.intracellular_network_condition["PMF,MF2"].to_csv("py_intra_network_PMF.csv")

In [25]:
def add_node_type(df):
    
    df['gene_A'] = np.where(df['type_gene_A'] == 'Ligand', df['gene_A'] + '|L', df['gene_A'])
    df['gene_A'] = np.where(df['type_gene_A'] == 'Receptor', df['gene_A'] + '|R', df['gene_A'])
    df['gene_A'] = np.where(df['type_gene_A'] == 'Transcription Factor', df['gene_A'] + '|TF', df['gene_A'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Ligand', df['gene_B'] + '|L', df['gene_B'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Receptor', df['gene_B'] + '|R', df['gene_B'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Transcription Factor', df['gene_B'] + '|TF', df['gene_B'])
    return df


In [26]:
def combine_LR_and_TF(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    #fix this
    lr_table = read_csv(LR_prediction)
    lr_table.rownames = lr_table.columns["X"]
    lr_table.columns["X"] = None

  intra_connections = pd.DataFrame()
  for celltype in np.unique([lr_table["source"], lr_table["target"]]):
    lr_filtered_ligands = lr_table[lr_table["source"] == celltype]
    lr_filtered_receptors = lr_table[lr_table["target"] == celltype]
    lr_ligands = np.unique(lr_filtered_ligands["gene_A"])
    lr_receptors = np.unique(lr_filtered_receptors["gene_B"])
    
    tf_table_receptors = tf_table[(tf_table["target"] == celltype) & (tf_table["type_gene_A"] == "Receptor")]
    tf_table_ligands = tf_table[(tf_table["source"] == celltype) & (tf_table["type_gene_B"] == "Ligand")]

    tf_receptor_interactions =  tf_table_receptors[tf_table_receptors["gene_A"].isin(lr_receptors)]
    tf_ligand_interactions = tf_table_ligands[tf_table_ligands["gene_B"].isin(lr_ligands)]

    intra_connections = pd.concat([intra_connections, tf_receptor_interactions, tf_ligand_interactions])
  intra_connections["all_pair"] = (intra_connections["source"] + "/" 
                                    + intra_connections["gene_A"] + "/"
                                    + intra_connections["target"] + "/"
                                    + intra_connections["gene_B"])
    
  intra_connections = intra_connections.drop_duplicates(subset=["all_pair"])
  intra_connections.drop(columns=["all_pair"], inplace=True)

  complete_interactions = pd.concat([intra_connections, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=0)
  return(complete_interactions)

In [48]:
def combine_LR_and_TF_unfiltered(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    #fix this
    lr_table = read_csv(LR_prediction)
    lr_table.rownames = lr_table.columns["X"]
    lr_table.columns["X"] = None


  complete_interactions = pd.concat([tf_table, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + "_unfiltered.csv"), index=0)
  return(complete_interactions)

In [ ]:
#ask for list with complexes to try?

def combine_LR_and_TF_complexes(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    #fix this
    lr_table = read_csv(LR_prediction)
    lr_table.rownames = lr_table.columns["X"]
    lr_table.drop(columns=["X"], inplace=True)

  intra_connections = pd.DataFrame()
  for celltype in np.unique([lr_table["source"], lr_table["target"]]):
    lr_filtered_ligands = lr_table[lr_table["source"] == celltype]
    lr_filtered_receptors = lr_table[lr_table["target"] == celltype]
    lr_ligands = np.unique(lr_filtered_ligands["gene_A"])
    lr_receptors = np.unique(lr_filtered_receptors["gene_B"])

    lr_receptors = pd.Series(lr_receptors)
    contains_complex = lr_receptors.str.contains("_", na=False)
    
    R_with_complex = lr_receptors[contains_complex]
    R_without_complex = lr_receptors[(~contains_complex)]
  
    tf_table_receptors = tf_table[(tf_table["target"] == celltype) & (tf_table["type_gene_A"] == "Receptor")]
 
    tf_receptor_interactions =  tf_table_receptors[tf_table_receptors["gene_A"].isin(R_without_complex)]
    
    complex_df = pd.DataFrame()
    if len(R_with_complex) > 0:
      for complex in R_with_complex:
        receptors = complex.split("_")
        R_TF_with_complex = tf_table_receptors[tf_table_receptors["gene_A"].isin(receptors)]
    
        if len(R_TF_with_complex) == 0:
          continue
        
        R_TF_with_complex.drop_duplicates()
        R_TF_with_complex["gene_A"] = complex
        
        complex_df = pd.concat([complex_df, R_TF_with_complex])

      complex_df.drop_duplicates()

    tf_receptor_interactions = pd.concat([tf_receptor_interactions, complex_df])
    print(tf_receptor_interactions)
    
    tf_table_ligands = tf_table[(tf_table["source"] == celltype) & (tf_table["type_gene_B"] == "Ligand")]

    
    tf_ligand_interactions = tf_table_ligands[tf_table_ligands["gene_B"].isin(lr_ligands)]

    intra_connections = pd.concat([intra_connections, tf_receptor_interactions, tf_ligand_interactions])
  intra_connections["all_pair"] = (intra_connections["source"] + "/" 
                                    + intra_connections["gene_A"] + "/"
                                    + intra_connections["target"] + "/"
                                    + intra_connections["gene_B"])
  print(intra_connections)
  intra_connections = intra_connections.drop_duplicates(subset=["all_pair"])
  intra_connections.drop(columns=["all_pair"], inplace=True)

  complete_interactions = pd.concat([intra_connections, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=0)
  return(complete_interactions)

In [27]:
#WINDOWS
table_ctr = pd.read_csv("LR2TF_test_run\\CTR_LR.csv")
table_exp = pd.read_csv("LR2TF_test_run\\EXP_LR.csv")

ctr_input = combine_LR_and_TF(result.CTR_input_condition["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF(result.CTR_input_condition["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")

In [98]:
#LINUX
table_ctr = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/CTR_LR.csv")
table_exp = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/EXP_LR.csv")

ctr_input = combine_LR_and_TF(result.CTR_input_condition["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF(result.CTR_input_condition["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")